In [ ]:
import os
import numpy as np
import cv2  # Import OpenCV for resizing
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-11-29 16:16:38.453116: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732878998.465346   36164 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732878998.469045   36164 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-29 16:16:38.481702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Clear Keras session
tf.keras.backend.clear_session()
# Force garbage collection to release memory
gc.collect()

0

In [ ]:
def preprocess_audio_dataset(input_folder, target_sr=16000, target_length=80000, n_mels=40, n_fft=1024, hop_length=512, top_db=20):
    spectrograms = []
    labels = []

    # Iterate over each audio file in the input folder
    for filename in os.listdir(input_folder):

        if filename.endswith((".wav", ".mp3", ".webm")):

            file_path = os.path.join(input_folder, filename)
            # print(f"Processing {filename}")

            # Step 1: Load and resample audio
            y, sr = librosa.load(file_path, sr=target_sr)

            # Step 2: Trim or pad the audio to the target length (5 seconds)
            y = trim_or_pad_audio(y, target_length)

            # Step 3: Remove silence from the audio
            y = remove_silence(y, sr, top_db)

            # Step 4: Generate a log mel-spectrogram
            log_mel_spec = generate_mel_spectrogram(y, sr, n_mels, n_fft, hop_length)

            # Step 5: Resize the spectrogram to 432x288 pixels
            log_mel_spec_resized = resize_spectrogram(log_mel_spec)

            # Store the spectrogram and corresponding label 
            spectrograms.append(log_mel_spec_resized)
            labels.append(filename)
            
        # break
    return spectrograms, labels

def trim_or_pad_audio(y, target_length=80000):
    if len(y) > target_length:
        return y[:target_length]  # Trim to the target length
    else:
        return np.pad(y, (0, target_length - len(y)), 'constant')  # Pad with zeros

def remove_silence(y, sr, top_db=20):
    return librosa.effects.trim(y, top_db=top_db)[0]

def generate_mel_spectrogram(y, sr, n_mels=40, n_fft=1024, hop_length=512):
    mel_spec = librosa.feature.melspectrogram(
        y=y, 
        sr=sr, 
        n_mels=n_mels, 
        n_fft=n_fft, 
        hop_length=hop_length,
        fmin=20, 
        fmax=8000
    )
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return log_mel_spec

def resize_spectrogram(spec, target_size=(224, 224)): # (224, 224)
    # Resize the spectrogram to the desired size using OpenCV
    return cv2.resize(spec, target_size, interpolation=cv2.INTER_LINEAR)

def display_spectrogram(spectrogram, title='Spectrogram'):
    plt.figure(figsize=(10, 6))
    librosa.display.specshow(spectrogram, sr=16000, x_axis='time', y_axis='mel', fmin=20, fmax=8000, cmap='coolwarm')
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.tight_layout()
    plt.show()

# Normalize function based on ImageNet stats (mean and std)
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

def preprocess_spectrogram(spectrogram):
    # Assuming spectrogram is a NumPy array of shape (224, 224)
    # Convert spectrogram from shape (224, 224) to (224, 224, 3)
    
    # First, normalize the spectrogram if necessary (e.g., if pixel range is [0, 255])
   
    spectrogram = spectrogram.astype(np.float16) / 255.0
    
    # Replicate the spectrogram into 3 channels (to match ResNet50's expected input shape)
    spectrogram = np.repeat(spectrogram[..., np.newaxis], 3, axis=-1) # Adds 3rd dimension  3
    
    # Normalize based on ImageNet stats
   
    spectrogram = (spectrogram - mean) / std
    
    return spectrogram

In [ ]:
# Preprocess the dataset from train folder
#train_folder = 'data/zipped/dataset_pt1(1)/dataset_pt1/train'
# test_folder = 'data/actual/test'

train_folder = 'data/first_two/train'
# test_folder = 'data/first_two/test'

# train_folder = 'data/last_two/train'
# test_folder = 'data/last_two/test'

# Example for one language folder, say 'english' in the train folder
language_1 = 'punjabi' 
train_path_1 = os.path.join(train_folder, language_1)
#test_path_1 = os.path.join(test_folder, language_1)

language_2 = 'pushto' 
train_path_2 = os.path.join(train_folder, language_2)
#test_path_2 = os.path.join(test_folder, language_2)

language_3 = 'saraiki' 
train_path_3 = os.path.join(train_folder, language_3)
#test_path_3 = os.path.join(test_folder, language_3)

language_4 = 'sindhi' 
train_path_4 = os.path.join(train_folder, language_4)
#test_path_4 = os.path.join(test_folder, language_4)

language_5 = 'urdu'  
train_path_5 = os.path.join(train_folder, language_5)
#test_path_5 = os.path.join(test_folder, language_5)

# Preprocess the train dataset for the language
punjabi_spectrograms, punjabi_labels = preprocess_audio_dataset(train_path_1)
pushto_spectrograms, pushto_labels = preprocess_audio_dataset(train_path_2)
saraiki_spectrograms, saraiki_labels = preprocess_audio_dataset(train_path_3)
sindhi_spectrograms, sindhi_labels = preprocess_audio_dataset(train_path_4)
urdu_spectrograms, urdu_labels = preprocess_audio_dataset(train_path_5)

train_spectrograms = punjabi_spectrograms + sindhi_spectrograms + pushto_spectrograms + saraiki_spectrograms + urdu_spectrograms

train_labels = ['punjabi'] * len(punjabi_spectrograms) + \
         ['sindhi'] * len(sindhi_spectrograms) + \
         ['pushto'] * len(pushto_spectrograms) + \
         ['saraiki'] * len(saraiki_spectrograms) + \
         ['urdu'] * len(urdu_spectrograms)

assert len(train_spectrograms) == len(train_labels), "Mismatch between spectrograms and labels count."

print("Total Spectrograms: ", len(train_spectrograms))

# Display one spectrogram as a sample
# display_spectrogram(train_spectrograms[0], title="train Spectrogram")

Total Spectrograms:  10000


In [ ]:
#del punjabi_spectrograms,sindhi_spectrograms,pushto_spectrograms,saraiki_spectrograms,urdu_spectrograms
del saraiki_labels,urdu_labels,punjabi_labels,sindhi_labels,pushto_labels
del sindhi_spectrograms,punjabi_spectrograms,pushto_spectrograms,urdu_spectrograms,saraiki_spectrograms
del train_folder
del train_path_1,train_path_2,train_path_3,train_path_4,train_path_5
gc.collect()

0

In [ ]:
# Preprocess training and testing spectrograms
train_spectrograms = np.array([preprocess_spectrogram(s) for s in train_spectrograms])
#test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape
print(f"Train Spectrograms Shape: {train_spectrograms.shape}")
#print(f"Test Spectrograms Shape: {test_spectrograms.shape}")

Train Spectrograms Shape: (10000, 224, 224, 3)


In [ ]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)  # Fit and transform the training labels

# Print label mapping for reference
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

train_spectrograms_, val_spectrograms, train_labels_encoded, val_labels_encoded = train_test_split(
    train_spectrograms, train_labels_encoded, test_size=0.2, random_state=42
)

# 1. Create TensorFlow Dataset objects for train and test datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_spectrograms_, train_labels_encoded))
val_dataset = tf.data.Dataset.from_tensor_slices((val_spectrograms, val_labels_encoded))

# Shuffle and batch the train dataset
train_dataset = train_dataset.shuffle(len(train_spectrograms_)).batch(64) # 64

# Batch the validation and test datasets (no shuffle needed)
val_dataset = val_dataset.batch(64) # 64

# 4. Inspect the dataset (first batch example)
for images, labels in train_dataset.take(1):  # Just take 1 batch to inspect
    print("Batch image shape:", images.shape)
    print("Batch labels shape:", labels.shape)

Label Mapping: {np.str_('punjabi'): np.int64(0), np.str_('pushto'): np.int64(1), np.str_('saraiki'): np.int64(2), np.str_('sindhi'): np.int64(3), np.str_('urdu'): np.int64(4)}


I0000 00:00:1732879098.055351   36164 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22979 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5
2024-11-29 16:18:18.056222: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9633792000 exceeds 10% of free system memory.
2024-11-29 16:18:22.049293: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9633792000 exceeds 10% of free system memory.
2024-11-29 16:18:25.609085: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2408448000 exceeds 10% of free system memory.
2024-11-29 16:18:26.844607: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2408448000 exceeds 10% of free system memory.
2024-11-29 16:18:27.690430: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 9633792000 exceeds 10% of free system memory.


Batch image shape: (64, 224, 224, 3)
Batch labels shape: (64,)


2024-11-29 16:18:30.328253: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
del train_spectrograms,train_spectrograms_,train_labels,train_labels_encoded

In [9]:
del val_labels_encoded,val_spectrograms

In [10]:
len(val_dataset)

32

In [ ]:
# Define the ResNet50 Model
base_model = tf.keras.applications.ResNet50(
    weights=None,#resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',  # Load weights manually
    input_shape=(224, 224, 3),
    include_top=False
)

# Manually load the weights
#base_model.load_weights('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')  

# Freeze the base model
base_model.trainable = True

# Add custom layers for classification
model = models.Sequential([
    base_model,                      # Pre-trained ResNet50
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),     # Normalize activations
    layers.Dense(256, activation='relu'),  # Dense layer
    layers.Dropout(0.5),             # Dropout for regularization
    layers.BatchNormalization(),     
    layers.Dense(128, activation='relu'),  # Dense layer
    layers.Dropout(0.5),             
    layers.BatchNormalization(),     
    layers.Dense(5, activation='softmax')  # Output layer for 5 classes
])

#     # Compile the model
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

# # Summary of the model
model.summary()
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=7,  # Stop after 2 epochs without improvement
    restore_best_weights=True
    )
    
class BatchLossLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.batch_losses = []
        
    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])

batch_logger1 = BatchLossLogger()

lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

checkpoint = ModelCheckpoint(
filepath='best_resnetmodel_pt1.keras',  # Save file path
monitor='val_loss',       # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
save_best_only=True,      # Save only the model with the best performance
mode='min',               # Mode for monitoring ('min' for loss, 'max' for accuracy)
verbose=1                 # Verbosity level (1 to print saving logs)
)

# Train the model
history1=model.fit(
train_dataset,  # Replace with the 4th folder's training dataset
validation_data=val_dataset,  # Replace with the 4th folder's validation dataset
epochs=50,  # Reduced maximum number of epochs
callbacks=[early_stopping,checkpoint,batch_logger1 ,lr_schedule
],
verbose=2
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,155,525 (92.15 MB)

 Trainable params: 24,097,541 (91.92 MB)

 Non-trainable params: 57,984 (226.50 KB)

Epoch 1/50


I0000 00:00:1732879131.554788   36328 service.cc:148] XLA service 0x711b440054f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732879131.554805   36328 service.cc:156]   StreamExecutor device (0): NVIDIA TITAN RTX, Compute Capability 7.5
2024-11-29 16:18:52.184149: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732879134.510771   36328 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-11-29 16:18:56.610328: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[64,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[64,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"


Epoch 1: val_loss improved from inf to 1.90585, saving model to best_resnetmodel_pt1.keras
125/125 - 71s - 567ms/step - accuracy: 0.3652 - loss: 1.6115 - val_accuracy: 0.1965 - val_loss: 1.9058 - learning_rate: 1.0000e-03
Epoch 2/50

Epoch 2: val_loss did not improve from 1.90585
125/125 - 27s - 217ms/step - accuracy: 0.4988 - loss: 1.2012 - val_accuracy: 0.1965 - val_loss: 3.0420 - learning_rate: 1.0000e-03
Epoch 3/50

Epoch 3: val_loss did not improve from 1.90585
125/125 - 27s - 218ms/step - accuracy: 0.5562 - loss: 1.0872 - val_accuracy: 0.1975 - val_loss: 3.8834 - learning_rate: 1.0000e-03
Epoch 4/50

Epoch 4: val_loss improved from 1.90585 to 1.68562, saving model to best_resnetmodel_pt1.keras
125/125 - 28s - 227ms/step - accuracy: 0.6126 - loss: 0.9455 - val_accuracy: 0.2465 - val_loss: 1.6856 - learning_rate: 1.0000e-03
Epoch 5/50

Epoch 5: val_loss did not improve from 1.68562
125/125 - 27s - 219ms/step - accuracy: 0.6501 - loss: 0.8763 - val_accuracy: 0.3860 - val_loss: 3.52

In [ ]:
del val_dataset,train_dataset

In [ ]:
# Preprocess the dataset from train folder
# train_folder = 'data/actual/train'
# test_folder = 'data/actual/test'

train_folder_2 = 'data/last_two/train'
test_folder_2 = 'data/last_two/test'

# Example for one language folder, say 'english' in the train folder
language_1 = 'punjabi' 
train_path_12 = os.path.join(train_folder_2, language_1)
#test_path_1 = os.path.join(test_folder, language_1)

language_2 = 'pushto' 
train_path_22 = os.path.join(train_folder_2, language_2)
#test_path_2 = os.path.join(test_folder, language_2)

language_3 = 'saraiki' 
train_path_32 = os.path.join(train_folder_2, language_3)
#test_path_3 = os.path.join(test_folder, language_3)

language_4 = 'sindhi' 
train_path_42 = os.path.join(train_folder_2, language_4)
#test_path_4 = os.path.join(test_folder, language_4)

language_5 = 'urdu'  
train_path_52 = os.path.join(train_folder_2, language_5)
#test_path_5 = os.path.join(test_folder, language_5)


# Preprocess the train dataset for the language
punjabi_spectrograms_2, punjabi_labels = preprocess_audio_dataset(train_path_12)
pushto_spectrograms_2, pushto_labels = preprocess_audio_dataset(train_path_22)
saraiki_spectrograms_2, saraiki_labels = preprocess_audio_dataset(train_path_32)
sindhi_spectrograms_2, sindhi_labels = preprocess_audio_dataset(train_path_42)
urdu_spectrograms_2, urdu_labels = preprocess_audio_dataset(train_path_52)

train_spectrograms_2 = punjabi_spectrograms_2 + sindhi_spectrograms_2 + pushto_spectrograms_2 + saraiki_spectrograms_2 + urdu_spectrograms_2

train_labels_2 = ['punjabi'] * len(punjabi_spectrograms_2) + \
         ['sindhi'] * len(sindhi_spectrograms_2) + \
         ['pushto'] * len(pushto_spectrograms_2) + \
         ['saraiki'] * len(saraiki_spectrograms_2) + \
         ['urdu'] * len(urdu_spectrograms_2)

assert len(train_spectrograms_2) == len(train_labels_2), "Mismatch between spectrograms and labels count."

print("Total Spectrograms: ", len(train_spectrograms_2))
train_spectrograms_2 = np.array([preprocess_spectrogram(s) for s in train_spectrograms_2])
#test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape
print(f"Train Spectrograms Shape: {train_spectrograms_2.shape}")
# Display one spectrogram as a sample
# display_spectrogram(train_spectrograms[0], title="train Spectrogram")

Total Spectrograms:  10000
Train Spectrograms Shape: (10000, 224, 224, 3)


In [14]:
#del punjabi_spectrograms,sindhi_spectrograms,pushto_spectrograms,saraiki_spectrograms,urdu_spectrograms
del saraiki_labels,urdu_labels,punjabi_labels,sindhi_labels,pushto_labels
del sindhi_spectrograms_2,punjabi_spectrograms_2,pushto_spectrograms_2,urdu_spectrograms_2,saraiki_spectrograms_2
del train_folder_2,test_folder_2 
del train_path_12,train_path_22,train_path_32,train_path_42,train_path_52

In [ ]:
# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded_2 = label_encoder.fit_transform(train_labels_2)  # Fit and transform the training labels

# # Print label mapping for reference
print("Label Mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

train_spectrograms_2_, val_spectrograms_2, train_labels_encoded_2, val_labels_encoded_2 = train_test_split(
    train_spectrograms_2, train_labels_encoded_2, test_size=0.2, random_state=42
)

# 1. Create TensorFlow Dataset objects for train and test datasets
train_dataset_2 = tf.data.Dataset.from_tensor_slices((train_spectrograms_2_, train_labels_encoded_2))
val_dataset_2 = tf.data.Dataset.from_tensor_slices((val_spectrograms_2, val_labels_encoded_2))

# Shuffle and batch the train dataset
train_dataset_2 = train_dataset_2.shuffle(len(train_spectrograms_2_)).batch(64)

# Batch the validation and test datasets (no shuffle needed)
val_dataset_2 = val_dataset_2.batch(644)

# 4. Inspect the dataset (first batch example)
for images, labels in train_dataset_2.take(1):  # Just take 1 batch to inspect
    print("Batch image shape:", images.shape)
    print("Batch labels shape:", labels.shape)

Label Mapping: {np.str_('punjabi'): np.int64(0), np.str_('pushto'): np.int64(1), np.str_('saraiki'): np.int64(2), np.str_('sindhi'): np.int64(3), np.str_('urdu'): np.int64(4)}
Batch image shape: (64, 224, 224, 3)
Batch labels shape: (64,)


2024-11-29 16:40:06.415659: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [16]:
del train_labels_encoded_2,train_spectrograms_2_
del val_labels_encoded_2,val_spectrograms_2
del images,labels

In [ ]:
model = load_model('best_resnetmodel_pt1.keras')

# Load the model trained on the 3rd folder
lr_schedule_2 = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
early_stopping_2 = EarlyStopping(
    monitor='val_loss',
    patience=7,  # Stop after 2 epochs without improvement
    restore_best_weights=True
    )
    
class BatchLossLogger(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.batch_losses = []
        
    def on_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])

batch_logger_2 = BatchLossLogger()
checkpoint_2 = ModelCheckpoint(
filepath='best_resnetmodel_pt2.keras',  # Save file path
monitor='val_loss',       # Metric to monitor (e.g., 'val_loss' or 'val_accuracy')
save_best_only=True,      # Save only the model with the best performance
mode='min',               # Mode for monitoring ('min' for loss, 'max' for accuracy)
verbose=1                 # Verbosity level (1 to print saving logs)
)

# Train the model
model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

history2=model.fit(
train_dataset_2,  # Replace with the 4th folder's training dataset
validation_data=val_dataset_2,  # Replace with the 4th folder's validation dataset
epochs=50,  # Reduced maximum number of epochs
callbacks=[early_stopping_2,checkpoint_2,batch_logger_2,lr_schedule_2
],
verbose=2
)

Epoch 1/50


2024-11-29 16:41:09.545545: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.20GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-11-29 16:41:09.945442: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=2} for conv (f32[644,64,56,56]{3,2,1,0}, u8[0]{0}) custom-call(f32[644,64,56,56]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-29 16:41:10.997668: W external/local_xla/xla/tsl/framework

ResourceExhaustedError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/mi08440/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/mi08440/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/mi08440/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/mi08440/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/mi08440/.local/lib/python3.10/site-packages/ipykernel/kernelbase.

In [ ]:
del train_dataset_2,val_dataset_2,checkpoint_2,early_stopping_2

In [ ]:
combined_loss = history1.history['loss'] + history2.history['loss']
combined_val_loss = history1.history['val_loss'] + history2.history['val_loss']
#96 train acc 80 valid acc for best model pt1 : its bestmodelpt2
#97 train acc 71 valid acc cnnpt2

In [ ]:
epochs = list(range(1, len(combined_loss) + 1))

plt.figure(figsize=(10, 6))
plt.plot(epochs, combined_loss, label='Training Loss')
plt.plot(epochs, combined_val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss vs. Epochs')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Combine batch losses
combined_batch_losses = batch_logger1.batch_losses + batch_logger_2.batch_losses
steps = list(range(1, len(combined_batch_losses) + 1))

plt.figure(figsize=(10, 6))
plt.plot(steps, combined_batch_losses, label='Loss Per Step')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Loss vs. Steps')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
del batch_logger1,batch_logger_2    
del combined_batch_losses,combined_loss,combined_val_loss
del epochs,history1,history2,language_1,language_2,language_3,language_4,language_5,steps

In [ ]:
# Preprocess the dataset from train folder

test_folder = 'data/first_two/test'
test_folder_2 = 'data/last_two/test'

# Example for one language folder, say 'english' in the train folder
language_1 = 'punjabi' 

test_path_1 = os.path.join(test_folder, language_1)
test_path_12 = os.path.join(test_folder_2, language_1)


language_2 = 'pushto' 

test_path_2 = os.path.join(test_folder, language_2)
test_path_22 = os.path.join(test_folder_2, language_2)

language_3 = 'saraiki' 

test_path_3 = os.path.join(test_folder, language_3)
test_path_32 = os.path.join(test_folder_2, language_3)

language_4 = 'sindhi' 

test_path_4 = os.path.join(test_folder, language_4)
test_path_42 = os.path.join(test_folder_2, language_4)

language_5 = 'urdu'  

test_path_5 = os.path.join(test_folder, language_5)
test_path_52 = os.path.join(test_folder_2, language_5)

# Preprocess the test dataset for the language
test_punjabi_spectrograms, test_punjabi_labels = preprocess_audio_dataset(test_path_1)
test_pushto_spectrograms, test_pushto_labels = preprocess_audio_dataset(test_path_2)
test_saraiki_spectrograms, test_saraiki_labels = preprocess_audio_dataset(test_path_3)
test_sindhi_spectrograms, test_sindhi_labels = preprocess_audio_dataset(test_path_4)
test_urdu_spectrograms, test_urdu_labels = preprocess_audio_dataset(test_path_5)

test2_punjabi_spectrograms, test2_punjabi_labels = preprocess_audio_dataset(test_path_12)
test2_pushto_spectrograms, test2_pushto_labels = preprocess_audio_dataset(test_path_22)
test2_saraiki_spectrograms, test2_saraiki_labels = preprocess_audio_dataset(test_path_32)
test2_sindhi_spectrograms, test2_sindhi_labels = preprocess_audio_dataset(test_path_42)
test2_urdu_spectrograms, test2_urdu_labels = preprocess_audio_dataset(test_path_52)


test_spectrograms = test_punjabi_spectrograms+ test2_punjabi_spectrograms+ test_sindhi_spectrograms+test2_sindhi_spectrograms + test_pushto_spectrograms +test2_pushto_spectrograms+ test_saraiki_spectrograms+test2_saraiki_spectrograms  + test_urdu_spectrograms+test2_urdu_spectrograms 

test_labels = [0] * len(test_punjabi_spectrograms+test2_punjabi_spectrograms) + \
         [3] * len(test_sindhi_spectrograms+test2_sindhi_spectrograms) + \
         [1] * len(test_pushto_spectrograms+test2_pushto_spectrograms) + \
         [2] * len(test_saraiki_spectrograms+test2_saraiki_spectrograms ) + \
         [4] * len(test_urdu_spectrograms+test2_urdu_spectrograms )
assert len(test_spectrograms) == len(test_labels), "Mismatch between spectrograms and labels count."



print("Total Spectrograms: ", len(test_spectrograms))
print("Total Labels: ", len(test_labels))
test_spectrograms = np.array([preprocess_spectrogram(s) for s in test_spectrograms])

# Ensure spectrograms are the correct shape

print(f"Test Spectrograms Shape: {test_spectrograms.shape}")

In [ ]:
del language_1,language_2,language_3,language_4,language_5
del test_folder,test_folder_2
del test_path_1,test_path_12,test2_urdu_spectrograms,test2_punjabi_labels,test2_pushto_labels
del test_path_2,test_path_22,test_path_3,test_path_32,test_path_4,test_path_42,test_path_5,test_path_52

In [ ]:
def evaluate_entire_testset(model, test_spectrograms, test_labels, labels_class):
    """
    Evaluates the model on the entire test set and computes the accuracy, confusion matrix,
    and a classification report.

    Args:
        model: The trained model to evaluate.
        test_spectrograms: Array of test spectrograms.
        test_labels: Array of corresponding test labels.
        labels_class: List of all possible class labels (e.g., [0, 1, 2, 3, 4]).

    Returns:
        overall_accuracy: The accuracy of the model on the test set.
        cm: The confusion matrix for the test set.
    """
    # Convert test data into a TensorFlow Dataset
    test_dataset = tf.data.Dataset.from_tensor_slices((test_spectrograms, test_labels))
    test_dataset = test_dataset.batch(32)  # Batch size

    # Predictions and true labels
    predictions = []
    true_labels = []

    for images, labels in test_dataset:
        # Predict the labels
        preds = model.predict(images)
        predicted_labels = np.argmax(preds, axis=1)  # Get the predicted class
        predictions.extend(predicted_labels)
        true_labels.extend(labels.numpy())  # Convert true labels to numpy array

    # Compute overall accuracy
    overall_accuracy = accuracy_score(true_labels, predictions)
    print(f"Overall Accuracy: {overall_accuracy:.4f}")

    # Compute the confusion matrix
    cm = confusion_matrix(true_labels, predictions, labels=labels_class)

    # Display the confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels_class, yticklabels=labels_class)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix for Entire Test Set')
    plt.show()

    # Print the classification report
    report = classification_report(true_labels, predictions, target_names=[f'Class {label}' for label in labels_class])
    print("Classification Report:")
    print(report)

    return overall_accuracy, cm


# Example usage
model = load_model('best_resnetmodel_pt2.keras')  # Load your model

# Assuming `test_spectrograms` and `test_labels` are already defined
labels_class = [0, 1, 2, 3, 4]  # All possible classes
overall_accuracy, cm = evaluate_entire_testset(model, test_spectrograms, test_labels, labels_class)


In [ ]:
# Assuming test_dataset is your tf.data.Dataset, and the model is already trained
test_dataset = tf.data.Dataset.from_tensor_slices((test_spectrograms, test_labels))
test_dataset = test_dataset.batch(16)  
# Initialize lists to store the true labels and predicted probabilities
true_labels = []
pred_probs = []

# Iterate over the test dataset
for spectrograms, labels in test_dataset:
    # Predict probabilities for the batch
    batch_pred_probs = model.predict(spectrograms)
    
    # Store the true labels and predicted probabilities
    true_labels.append(labels.numpy())  # Convert Tensor to numpy array
    pred_probs.append(batch_pred_probs)

# Convert lists to numpy arrays
true_labels = np.concatenate(true_labels, axis=0)
pred_probs = np.concatenate(pred_probs, axis=0)

# Binarize the true labels for ROC
true_labels_bin = label_binarize(true_labels, classes=[0, 1, 2, 3, 4])
n_classes = true_labels_bin.shape[1]

# Plot ROC curve
plt.figure(figsize=(10, 8))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(true_labels_bin[:, i], pred_probs[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

# Diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--')  # Diagonal line

# Add labels and title
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Test Dataset')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()


In [ ]:
del true_labels,true_labels_bin,tpr,spectrograms,roc_auc,pred_probs,overall_accuracy,n_classes,model,labels_class,labels,label_encoder,i,fpr,cm,batch_pred_probs